In [11]:
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer, util
import torch

In [21]:
df = pd.read_csv("./filtered_data.csv")
df

,old_code,fixed_code,SmellRule,smellKey_encoded
0,"{""function"": ""public boolean doSave(Carrello b...","{""function"": ""public boolean doSave(Carrello b...",java:S106\n,0
1,"{""function"": ""protected void doPost(HttpServle...","{""function"": ""protected void doPost(HttpServle...",java:S106\n,0
2,"{""function"": ""public static synchronized void ...","{""function"": ""public static synchronized void ...",java:S106\n,0
3,"{""function"": ""public static synchronized void ...","{""function"": ""public static synchronized void ...",java:S106\n,0
4,"{""function"": ""public static synchronized byte[...","{""function"": ""public static synchronized byte[...",java:S106\n,0
...,...,...,...,...
88,"{""function"": ""public Object deserialize(JsonPa...","{""function"": ""public Object deserialize(JsonPa...",java:S3740\n,23
89,"{""function"": "" CodeEmitter<T> emit(final Strin...","{""function"": "" CodeEmitter<T> emit(final Strin...",java:S3776\n,24
90,"{""function"": ""public Expression createExpressi...","{""function"": ""public Expression createExpressi...",java:S3740\n,23
91,"{""function"": ""public ConfigurationPropertiesVa...","{""function"": ""public ConfigurationPropertiesVa...",java:S3776\n,24


In [22]:
import csv
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import pickle
# Extract the "smellKey" column values
smellKey = df['SmellRule'].values

# Perform label encoding
encoder = LabelEncoder()
encoded_messages = encoder.fit_transform(smellKey)

# Update the DataFrame with the encoded values
df_encoded = df.copy()
df_encoded['smellKey_encoded'] = encoded_messages

# Save the encoded content to a new CSV file
output_file = 'Temp_smellKey-data_encoded.csv'
df_encoded.to_csv(output_file, index=False)

# Save the encoding vocabulary
vocabulary_file = 'smellKey_encoding_vocabulary.pkl'
with open(vocabulary_file, 'wb') as f:
    pickle.dump(encoder.classes_, f)


In [23]:
import pickle

# Extract the 'message_encoded' column
message_encoded = df['SmellRule']

# Save the message_encoded values to a pickle file
with open('SmellRuleEncoded.pkl', 'wb') as file:
    pickle.dump(message_encoded, file)
with open('SmellRuleEncoded.pkl', 'wb') as file:
    pickle.dump(message_encoded, file)

In [26]:
old_code = df['old_code'].tolist()
preprocessed_old_code = [fn.strip() for fn in old_code]

fixed_code = df['fixed_code'].tolist()
preprocessed_fixed_code = [fn.strip() for fn in fixed_code]

model = SentenceTransformer('bert-base-nli-mean-tokens')

old_embeddings = model.encode(preprocessed_old_code, convert_to_tensor=True)
save_path2 = 'bert_old2.pt'
torch.save(old_embeddings, save_path2)
fixed_embeddings = model.encode(preprocessed_fixed_code, convert_to_tensor=True)

save_path = 'bert_fixed2.pt'
torch.save(fixed_embeddings, save_path)


In [27]:
loaded_fixembed = torch.load(save_path)
loaded_fixembed

tensor([[-2.4374e-01,  1.0850e+00,  1.6211e-01,  ..., -5.7251e-01,
          1.9241e-01,  4.1632e-01],
        [ 6.8626e-04,  5.4104e-01,  5.9159e-01,  ..., -1.1429e-01,
         -4.1351e-01,  6.4309e-01],
        [ 1.1854e-01,  8.9044e-01,  2.8712e-01,  ..., -5.0341e-01,
          1.2426e-01,  4.8268e-01],
        ...,
        [-2.6283e-01,  1.1781e+00,  2.3453e-01,  ..., -1.3080e-01,
          9.9070e-02,  1.1359e-01],
        [-3.0876e-01,  2.9136e-01,  5.4072e-01,  ..., -1.3681e-01,
         -2.7539e-01,  6.3392e-01],
        [-3.5697e-01,  5.4462e-01,  8.1329e-01,  ..., -1.5163e-01,
         -3.1658e-01,  4.3520e-01]])

In [28]:
save_path2 = 'bert_old2.pt'
loaded_oldembed= torch.load(save_path2)
loaded_oldembed                                     

tensor([[-2.4374e-01,  1.0850e+00,  1.6211e-01,  ..., -5.7251e-01,
          1.9241e-01,  4.1632e-01],
        [ 6.8626e-04,  5.4104e-01,  5.9159e-01,  ..., -1.1429e-01,
         -4.1351e-01,  6.4309e-01],
        [ 1.1854e-01,  8.9044e-01,  2.8712e-01,  ..., -5.0341e-01,
          1.2426e-01,  4.8268e-01],
        ...,
        [-2.6283e-01,  1.1781e+00,  2.3453e-01,  ..., -1.3080e-01,
          9.9070e-02,  1.1359e-01],
        [-3.0876e-01,  2.9136e-01,  5.4072e-01,  ..., -1.3681e-01,
         -2.7539e-01,  6.3392e-01],
        [-3.5697e-01,  5.4462e-01,  8.1329e-01,  ..., -1.5163e-01,
         -3.1658e-01,  4.3520e-01]])

In [29]:
num_embeddings = loaded_oldembed.shape[0]
print(f"Number of old embeddings: {num_embeddings}")
num_embeddings = loaded_fixembed.shape[0]
print(f"Number of fix embeddings: {num_embeddings}")

Number of old embeddings: 93
Number of fix embeddings: 93
